In [24]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import requests
import os

Data

In [31]:

load_dotenv()
geo_key = os.getenv('GEO_CODE')

df = pd.read_csv('toronto_housing_data_03222021.json.csv')

df['Price'] = df['price'].str.replace(r'[^\d]', '', regex=True)
df['Bedrooms'] = df['details'].str.extract(r'(\d+) bd?s?')
df['Bathrooms'] = df['details'].str.extract(r'(\d+) ba')


def geocode_address(address, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': address,
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error: {data['status']} for address: {address}")
            return None, None
    else:
        print(f"Request failed: {response.status_code}")
        return None, None
    
df['Latitude'], df['Longitude'] = zip(*df['address'].apply(lambda x: geocode_address(x, geo_key)))
df.to_csv('geocoded_addresses.csv', index=False)
print("Geocoded data saved to 'geocoded_addresses.csv'.")




df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'])


df = df.drop(columns=['details', 'price'])

print(df)


TypeError: object of type 'zip' has no len()